## Loading Training data

In [1]:
# importing packages 
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import requests
import datetime
import torch
import torch.utils.data
import os
import importlib
import sys
import re
import pickle
from mpl_toolkits import mplot3d
from io import BytesIO
from math import log, exp, tan, atan, ceil
from PIL import Image
from utils import dataset_utils
from utils import createAISdata
#from utils import protobufDecoder
#from utils import plotting
from models import VRNN
from Config import config

# To measure the training time
from time import time

import pickle

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f">> Using device: {device}")
if device=="cuda:0":
    torch.no_grad()
    torch.cuda.empty_cache()
#timestamp = datetime.datetime.fromtimestamp(update.t_epoch_sec).strftime('%d/%m/%Y %H:%M:%S')

shiptypes = config.SHIPTYPE_CARGO + config.SHIPTYPE_TANKER
shipFileName = 'test'
binedges = (config.LAT_EDGES, config.LON_EDGES, config.SOG_EDGES, config.COG_EDGES)
batch_size = 4

>> Using device: cpu


In [3]:
#path = "C://Users//asm//OneDrive - Netcompany/University//Master Thesis//Data//Pickle//CargTank_1911.pkl"
#path_index = "C://Users//asm//OneDrive - Netcompany/University//Master Thesis//Data//Pickle//CargTank_1911_idxs.pkl"

##path = "C://Users//asm//OneDrive - Netcompany/University//Master Thesis//pickleFiles//CargTank.pkl"
##path_index = "C://Users//asm//OneDrive - Netcompany/University//Master Thesis//pickleFiles//CargTank_idxs.pkl"

path = "C://Users//asm//OneDrive - Netcompany/University//Master Thesis//Data//Pickle//"
path_index = "C://Users//asm//OneDrive - Netcompany/University//Master Thesis//Data//Pickle//CargTank_dxs.pkl"

datasets_path = "C://Users//asm//OneDrive - Netcompany/University//Master Thesis//Data//Pickle//"
#datasets_path = "C://Users//asm//OneDrive - Netcompany/University//Master Thesis//Data//Pickle//"

In [4]:
class PadSequence:
    def __call__(self, batch):
                
        # each element in "batch" is a tuple ( mmsis,  shiptypes,  lengths, inputs, targets)
        # Get each sequence and pad it
        mmsis = [x[0] for x in batch] # Maritime Mobile Service Identity numbers
        shiptypes = [x[1] for x in batch] # tank, cargo, etc.
        lengths = [x[2] for x in batch] # used as measure of size
        inputs = [x[3] for x in batch] # they are normalized 
        targets = [x[4] for x in batch] # seems to contain the real path of the vessel
                                        # lat, lon, speed, course (NOT NORMALIZED)
                
        inputs_padded = torch.nn.utils.rnn.pad_sequence(inputs, batch_first=True)
        targets_padded = torch.nn.utils.rnn.pad_sequence(targets, batch_first=True)

        return  torch.tensor(mmsis),  torch.tensor(shiptypes),  torch.tensor(lengths, dtype=torch.float), inputs_padded, targets_padded

In [5]:
# different lengths (use max/min for dimensions)
trainset = dataset_utils.AISDataset(dataPath = datasets_path, fileName = "CargTank.pkl")
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers = 0, collate_fn=PadSequence())

dataPath: C://Users//asm//OneDrive - Netcompany/University//Master Thesis//Data//Pickle//
fileName: CargTank.pkl
self.params[dataFileName]: CargTank_dxs.pkl


self.datapath 12 C://Users//asm//OneDrive - Netcompany/University//Master Thesis//Data//Pickle//CargTank_dxs.pkl
index:  22787567   total_updates:  643429


In [6]:
testset = dataset_utils.AISDataset(dataPath = datasets_path, fileName = "CargTank.pkl", train_mean = trainset.mean)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers = 0, collate_fn=PadSequence())

dataPath: C://Users//asm//OneDrive - Netcompany/University//Master Thesis//Data//Pickle//
fileName: CargTank.pkl
self.params[dataFileName]: CargTank_dxs.pkl


In [7]:
train_n = len(trainset)
test_n = len(testset)
num_batches = len(train_loader)
num_epochs = 1

In [8]:
print('Training set size is: {} and Test set size is {} '.format(train_n,test_n))

Training set size is: 10118 and Test set size is 2530 


In [9]:
trainset.maxLength

144

In [10]:
testset.maxLength

144

In [42]:
model = VRNN.VRNN(input_shape=trainset.datadim, latent_shape=config.LATENT_SIZE, generative_bias=trainset.mean, device=device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)

In [43]:
trainset.datadim

167

In [44]:
def computeLoss(log_px, log_pz, log_qz, lengths, beta=1):
    
    max_len = len(log_px)
    curmask = torch.arange(max_len, device=device)[:, None] < lengths[None, :] #max_seq_len X Batch
    
    log_px = torch.stack(log_px, dim=0) * curmask
    log_px = log_px.sum(dim=0) #Sum over time
   
    log_pz = torch.stack(log_pz, dim=0) * curmask
    log_qz = torch.stack(log_qz, dim=0) * curmask
    kl = log_qz.sum(dim=0) - log_pz.sum(dim=0) #Sum over time
    
    loss = log_px - beta * kl #recon loss - beta_kl
    loss = torch.mean(loss/lengths) #mean over batch
    
    return -loss, log_px, kl

In [45]:
loss_tot = []
kl_tot = []
recon_tot = []
val_loss_tot = []
val_kl_tot = []
val_recon_tot = []

In [46]:
trainset.maxTrackLen_train

75

In [47]:
testset.maxTrackLen_test

73

In [28]:
zmus = torch.zeros(trainset.maxTrackLen_train, train_n, config.LATENT_SIZE , device = 'cpu')

zmus.shape

torch.Size([75, 10118, 100])

In [31]:
loss_epoch = 0
kl_epoch = 0
recon_epoch = 0
model.train()

for i, (mmsi, label,speedBinLabel,courseBinLabel, lengths, inputs, targets) in enumerate(train_loader):
    
    inputs = inputs.to(device)
    targets = targets.to(device)
    lengths = lengths.to(device)

    log_px, log_pz, log_qz, _, _, z_mus = model(inputs,targets,logits=None)

    loss, log_px, kl = computeLoss(log_px, log_pz, log_qz, lengths)

    model.zero_grad()
    loss.backward()
    optimizer.step()

    loss_epoch += loss.item()*len(lengths)
    kl_epoch += torch.sum(kl/lengths).item()
    recon_epoch += torch.sum(log_px/lengths).item()

    z_means = {
        'Epoch': epoch,
        'z_means': z_mus,
        'labels': label,
        'log_px': log_px,
        'log_pz': log_pz,
        'log_qz': log_qz
        }
    
    print('z_mus.shape: ',z_mus.shape)

    #with open('models/saved_models/latentSpace_train.pkl', "wb") as f:
        #

    #    pickle.dump(z_means, f)

#loss_tot.append(loss_epoch/train_n)
#kl_tot.append(kl_epoch/train_n)
#recon_tot.append(recon_epoch/train_n)

#Begin validation loop
#val_loss = 0
#val_kl = 0
#val_recon = 0



IndexError: dimension specified as 0 but tensor has no dimensions

In [30]:
## Examining the training set

#i = 0

#for i, (_, _, lengths, inputs, targets) in enumerate(train_loader):
    
#    i = i +1

#print('Total: {}'.format(i))

#for i, (_, _, lengths, inputs, targets) in enumerate(train_loader):
    
    #print(lengths)
loss_epoch = 0
kl_epoch = 0
recon_epoch = 0
model.train()
z_mus_train = 0

#zmus = torch.zeros(trainset.maxTrackLen_train, train_n, config.LATENT_SIZE , device = 'cpu')

for i, (mmsi, label, lengths, inputs,speedBinLabel,courseBinLabel, targets) in enumerate(train_loader):
    inputs = inputs.to(device)
    targets = targets.to(device)
    lengths = lengths.to(device)
    
    #Get the maximum length of the current batch
    #max_len = int(torch.max(lengths).item())
    
    print('max_len ',max_len)
    
    log_px, log_pz, log_qz, _, _, z_mus = model(inputs,targets,label,logits=None)
    #print('label: {}'.format(label))
    z_mus_train = z_mus
    z_means = {
        'z_means': z_mus,
        'labels': label,
        'log_px': log_px,
        'log_pz': log_pz,
        'log_qz': log_qz
    }
    
    #print('z_mus.shape', z_mus.shape)
   # with open('models/saved_models/latentSpace_test.pkl', "wb") as f:
        
   #     pickle.dump(z_means, f)
        speedBinLabel = track['speedBinLabel']
        courseBinLabel = track['courseBinLabel']
        
   # print('----------------------------------')
    break
#    print('--------------------------------------------------------')

return  
torch.tensor(track['mmsi']), 
torch.tensor(label),
torch.tensor(speedBinLabel),
torch.tensor(courseBinLabel), 
torch.tensor(track['track_length'], dtype=torch.float), 
inputs, 
targets

IndexError: dimension specified as 0 but tensor has no dimensions

In [42]:
z_mus_train[:,3,:].shape

torch.Size([87, 100])

In [18]:
for epoch in range(1, num_epochs+1): #num_epochs+1
    #Begin training loop
    tic = time()

    loss_epoch = 0
    kl_epoch = 0
    recon_epoch = 0
    model.train()
    for i, (_, _, lengths, inputs, targets) in enumerate(train_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)
        lengths = lengths.to(device)
        
        log_px, log_pz, log_qz, _, _ = model(inputs,targets,logits=None)
        
        loss, log_px, kl = computeLoss(log_px, log_pz, log_qz, lengths)
        
        model.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss_epoch += loss.item()*len(lengths)
        kl_epoch += torch.sum(kl/lengths).item()
        recon_epoch += torch.sum(log_px/lengths).item()
    
    loss_tot.append(loss_epoch/train_n)
    kl_tot.append(kl_epoch/train_n)
    recon_tot.append(recon_epoch/train_n)
    
    #Begin validation loop
    val_loss = 0
    val_kl = 0
    val_recon = 0
    model.eval()
    for i, (_, _, lengths, inputs, targets) in enumerate(test_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)
        lengths = lengths.to(device)
        
        log_px, log_pz, log_qz, _, _ = model(inputs,targets,logits=None)
        
        loss, log_px, kl = computeLoss(log_px, log_pz, log_qz, lengths)
                
        val_loss += loss.item()*len(lengths)
        val_kl += torch.sum(kl/lengths).item()
        val_recon += torch.sum(log_px/lengths).item()
    
    val_loss_tot.append(val_loss/test_n)
    val_kl_tot.append(val_kl/test_n)
    val_recon_tot.append(val_recon/test_n)
    
    datapoints = np.random.choice(test_n, size = 3, replace=False)
    #plotting.make_vae_plots((loss_tot, kl_tot, recon_tot, val_loss_tot, val_kl_tot, val_recon_tot), model, datapoints, testset, binedges, device)
    
    #print('Epoch {} of {} finished. Trainingloss = {}. Validationloss = {}'.format(epoch, num_epochs, loss_epoch/train_n, val_loss/test_n))
    dataset_utils.eprint('Epoch {} of {} finished. Trainingloss = {}. Validationloss = {}'.format(epoch, num_epochs, loss_epoch/train_n, val_loss/test_n))

In [20]:
#index:  2777757   total_updates:  238708  totalRows:  238708
#index:  9727785   total_updates:  190279  totalRows:  190279

In [14]:
with open(path, "rb") as f:
    aa = pickle.load(f)

In [15]:
aa.keys()

dict_keys(['trainIndicies', 'testIndicies', 'dataFileName', 'ROI', 'timeperiod', 'maxspeed', 'navstatuses', 'shiptypes', 'binedges', 'minTrackLength', 'maxTrackLength', 'resampleFrequency'])

In [16]:
aa['shiptypes']

[70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89]

In [21]:
latentSpace = 'models/saved_models/latentSpace.pkl'

In [22]:
with open(latentSpace, "rb") as ff:
    bb = pickle.load(ff)

In [23]:
bb.keys()

dict_keys(['z_means', 'labels', 'log_px', 'log_pz', 'log_qz'])

In [27]:
bb['labels']

tensor([1, 0, 0, 1])

In [48]:
path1 = path + 'CargTank.pkl'

In [49]:
with open(path1, "rb") as ff:
    bb = pickle.load(ff)

In [50]:
bb.keys()

dict_keys(['trainIndicies', 'testIndicies', 'dataFileName', 'ROI', 'timeperiod', 'maxspeed', 'navstatuses', 'shiptypes', 'binedges', 'minTrackLength', 'maxTrackLength', 'resampleFrequency'])

In [87]:
indices = bb['trainIndicies'][:10]

In [94]:
def findMaxTrackLength(indices):
    
    ##
    trackLenList = [] 

    for index in indices:
        with open(path_index,'rb') as file:
            file.seek(index)
            track = pickle.load(file)
            trackLenList.append(track['track_length'])
    return max(trackLenList)

In [93]:
trackLenList + trackLenList

[103,
 69,
 76,
 36,
 59,
 52,
 68,
 49,
 58,
 80,
 103,
 69,
 76,
 36,
 59,
 52,
 68,
 49,
 58,
 80]

In [65]:
tmpdf

,mmsi,shiptype,track_length,lat,lon,speed,course,speedBinLabel,courseBinLabel,timestamp
0,244267000,89,103,55.0,15.333333,5.666667,252.666667,3,3,1.570796e+09
1,244267000,89,103,55.0,15.000000,5.428571,255.714286,3,3,1.570796e+09
2,244267000,89,103,55.0,15.000000,5.454545,254.454545,3,3,1.570797e+09
3,244267000,89,103,55.0,15.000000,5.804878,257.219512,3,3,1.570798e+09
4,244267000,89,103,55.0,15.000000,6.046512,256.604651,3,3,1.570798e+09
...,...,...,...,...,...,...,...,...,...,...
98,244267000,89,103,54.0,13.000000,9.928571,247.000000,3,3,1.570855e+09
99,244267000,89,103,54.0,13.000000,10.000000,249.062500,3,3,1.570855e+09
100,244267000,89,103,54.0,13.000000,10.000000,250.320000,3,3,1.570856e+09
101,244267000,89,103,54.0,13.000000,10.000000,250.217391,3,3,1.570856e+09


In [58]:
max(tmpdf['track_length'])

80